### Spotipy Workshop

### Step-by-step guide for Part 2
    * Generate a playlist based on your library and recent activity
        * Get ids of the tracks in your playlists (or recently played tracks)
        * Get musical features for those tracks
        * Order the tracks from "old" to "new" 
        * Do a multi-class classification
        * Get recommendations and do classification again
    * Do clustering of all the tracks in the library by:
        * popularity + audio features + is_local
        
##### acknoledgements go to https://github.com/smyrbdr/make-your-own-Spotify-playlist-of-playlist-recommendations/blob/master/Make_Your_Own_Playlist_of_Recs-with_PCA%2Btf-idf%2BDT_on_Blues.ipynb

In [1]:
SPOTIPY_CLIENT_ID = None
SPOTIPY_CLIENT_SECRET = None
SPOTIPY_REDIRECT_URI = 'http://localhost/'
SPOTIFY_USERNAME = None

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

from IPython.display import Image, Audio, display
from IPython.core.display import HTML 

from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
scope = 'user-library-read playlist-read-private user-read-recently-played user-top-read playlist-modify-public'

In [3]:
client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
token = util.prompt_for_user_token(SPOTIFY_USERNAME, scope)
if token:
    sp = spotipy.Spotify(auth=token)
else:
    print('Check the enviroment variables, the redirect URI and try again')

In case you have a lot of playlists, this step may take a lot of time. In this case, you may try to reduce the number of playlists.

In [ ]:
playlists = sp.current_user_playlists()['items']
playlist_df = list()
tracks = dict()

for playlist in playlists[:2]:
    playlist_data = sp.user_playlist(SPOTIFY_USERNAME, playlist['id'], fields='tracks')['tracks']['items']
    
    for track in playlist_data:
        if track['track']['id']: # Removes the local tracks in your playlist if there is any
            tracks[track['track']['id']] = track['track']['name']

    features = list()
    for track_id in tracks.keys():
        audio_features = sp.audio_features([track_id])
        print(audio_features)
        for track in audio_features:
            features.append(track)

    playlist_df.append(pd.DataFrame(features, index = tracks.keys()))

In [34]:
features_df = pd.concat(playlist_df)
features_df=features_df[["id", "acousticness", "danceability", "duration_ms", 
                         "energy", "instrumentalness",  "key", "liveness",
                         "loudness", "mode", "speechiness", "tempo", "valence"]]
features_df.head()

,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
7L1JkVqPNlVMoKfWIjRlGZ,7L1JkVqPNlVMoKfWIjRlGZ,0.505000,0.399,237200,0.7560,0.00246,2,0.0842,-7.870,1,0.0559,124.981,0.4330
2onqdFMfjOuSEA8s9kCMuc,2onqdFMfjOuSEA8s9kCMuc,0.000182,0.581,229773,0.9200,0.24300,7,0.2420,-6.779,0,0.0406,135.065,0.2300
7L1JkVqPNlVMoKfWIjRlGZ,7L1JkVqPNlVMoKfWIjRlGZ,0.505000,0.399,237200,0.7560,0.00246,2,0.0842,-7.870,1,0.0559,124.981,0.4330
2onqdFMfjOuSEA8s9kCMuc,2onqdFMfjOuSEA8s9kCMuc,0.000182,0.581,229773,0.9200,0.24300,7,0.2420,-6.779,0,0.0406,135.065,0.2300
4QTljisNteTZxV5rUBu9ji,4QTljisNteTZxV5rUBu9ji,0.991000,0.264,289533,0.0152,0.92000,1,0.0983,-31.280,1,0.0402,72.355,0.0444


**Here goes the missing part on labelling the data and training the classifier. The classifier should distinguis between the songs which you really like and the songs which happened to be in your library by accident.**

Next, we will extract **the recommendation from Spotify.** We can send a seed track and get a list of similar/recommended tracks for each seed.

In [43]:
rec_tracks = list()
for track_id in tracks.keys():
    recommended = sp.recommendations(seed_tracks=[track_id], limit=5)['tracks']
    rec_tracks += [track['id'] for track in recommended]
    print([(track['id'], track['name'], track['album']['artists'][0]['name']) for track in recommended])

[('6HZBLw5O3bowdv1JpNNDRw', 'Inion Ni Scannlain', 'Lúnasa'), ('3Lgq7X51hHSTJ8njTAlUmb', 'All Souls Night - Live', 'Loreena McKennitt'), ('1nd37vT8JNshn5Jrq0KN0M', 'Reel Around The Sun', 'Various Artists'), ('0wf136XdLOC4tMSnhTDabf', 'The Red Shoes', 'Natalie MacMaster'), ('32Lw7lCpUZsecY3gGeORxf', 'Cry Of The Celts', 'Ronan Hardiman')]
[('30X3fQ87XRzIilgWrYVBd4', 'Desperado From Outer Space', 'Eric Speed'), ('0qTb6APm0CI4DGFfLQJS20', 'Guiding Light', 'Simply Three'), ('0g1xBJGgVnL8tUEIoH0ng9', 'This Is What It Feels Like', 'Ely Jaffe'), ('0uFwdpkVJzpyUxlu9reTWK', "Love's Just a Feeling (feat. Rooty)", 'Lindsey Stirling'), ('6Zpn04HhXbuU9sLJQIY0bI', 'Rock the Ice', 'Edvin Marton')]
[('11Gj7oeUz27BskXHwDsWH1', 'Zelenu Granu s Tugom Zutog Voca', 'Klapa Cambi'), ('2MIVLgLJcfAKB8CGutZPhY', 'Gdje Si Majko Da Me Budis', 'Klapa Sveti Juraj HRM'), ('66LFlaSMlWNEIHbBbkzxhD', 'Hengameh', 'Patrick Mimran'), ('6pcIgvJnBC724thDZDdQsC', 'Sušna Zikva Na Škoju', 'Klapa Greben'), ('3vVb5JqcEqb1UdM24oYSS

[('210mEWCJ3rIexaoM25gwte', 'Crest of Honor', 'David R. Gillingham'), ('7kOKFeqXs0m7rCpedy9d8x', 'Tuba Concerto: I. Fiery and Bold', 'Donald Grantham'), ('2T2ggJGZ8epeBRqZAH98oi', 'チャンス: 呪文と踊り', 'Shigeo Genda'), ('4PCUSxpsgFe0WC01KnKLY8', 'Symphonic Suite: IV. Antique Dance', 'North Texas Wind Symphony'), ('2egQO3iabK0kMzAAmfKC83', 'Colours: V. Dark Ivy', 'William Berz')]
[('6YnLjTUvVYYB0nRzYohXEQ', 'Торжественный марш', 'Valery Gergiev'), ('09VUH9HE5QAdzcGnAFYhfN', "Le Sacre du printemps, Première Partie: L'Adoration de la terre: Jeu de rapt", 'Oslo Philharmonic OrchestraMariss Jansons'), ('7FTuhPHOD1pxg5awM4cXJ3', 'Il Barbiere di Siviglia (1983 Digital Remaster): Overture', 'Riccardo Muti'), ('3Nqm5OWwLyAdzUXhasubY4', 'Knuss-Olles livsstycke', 'Gävle Symphony Orchestra'), ('3v4eYvx9lmS1Im0pKHS1y0', 'The Power And The Glory', 'Slovak Radio Symphony Orchestra')]
[('4V71MJIdhBI2lqCSt3p5LO', "Duke Ellington's Sound of Love", 'Tülay German'), ('3q5nywOeVtUy4l4DBQB7UV', "İstanbul'u Dinliyo

[('0CQskMydQ8AhtKhJUAolvE', 'Tallis One', 'Samuel Lindon'), ('46zwIYfzF1zfG0w2txfWX4', 'Darkness Falls', 'Philip Wesley'), ('5ToS9stuUq9vIflZ8xnwdp', 'Vega', 'Olivia Belli'), ('5IUwKdwvmqINnWtu1Pesuq', 'Circles from the rue Simon-Crubellier', 'Jeroen van Veen'), ('7FsRo0AFkN8rtrGwkERjYK', 'Maud: Birds in the High Hall Garden (arr. R. Threlfall, J. Lenehan and J. Lloyd Webber for cello and piano)', 'Julian Lloyd Webber')]
[('5nfwAFSiJZOLXU2NrfRuKH', 'But the archer himself feeling the arrowâ\x80\x99s point from another bow', 'Wim Mertens'), ('6l3UgKcF3wfoqwQsOR3y56', "L'homme Noir", 'amiina'), ('7x8dzibKU1KGaxrnmdGvMG', 'Ephemeral', 'Akira Kosemura'), ('1utgYBI83vtaCQYfnqeGQA', 'Fine', "Dustin O'Halloran"), ('4QYMqouLpJ0M73zYnSZvkW', 'Oceano', 'Roberto Cacciapaglia')]
[('2eAW7rLQG8wLRpwgPYoORi', 'Turbulent Serenity - Instrumental', 'Anne Dudley'), ('76xjzR8fdW017mG8DTPdWa', 'Sorry Welby', 'Carter Burwell'), ('2kawDOYZbvd78Hld2cCohs', 'Hurricane', 'Slow Meadow'), ('1eIj2Dxi8yAhS2KAHdmqDA

[('0mim6nkzHTzujVP0XTmG1O', 'Eyjafjallajökull', 'Theodore Shapiro'), ('3mRn0Go2X1oePT8dfCWrBL', 'The Sicilian Clan (From "The Sicilian Clan")', 'Ennio Morricone'), ('2uoXFYabknBXA3NIXVPA70', 'Library of Congress', 'Trevor Rabin'), ('6wRmtN9ZWAYSMFFLa36wKc', 'Had My Doubts', 'Jeff Beal'), ('3ax24ycWdRrnwCDVlL5JSF', 'Get Your Tits Up (Suite)', 'Various Artists')]
[('3DupCTMbHX2vBNqhugOCph', 'New And Old', 'Louis Landon'), ('3x5GkSg9r6ShfiJf7wNnDe', "Adam's Lullabye", 'Tim Neumark'), ('6hMDOwWdSD6PBmDF9Z5OI6', 'Evening Lake, Live Music of Only Truth, Power of Stars in the Water, Crystal-Clear Understanding of Unique Purpose of Existence, Healing from Pain, Salvation from Sufferings', 'Andrei Krylov, Ana Svet & Shakti Ros'), ('05Bg2urbKQqwlaDI7gIFFN', 'Lighthouse', 'Thad Fiscella'), ('5n3HiIGBvqEJxXWU9kKYmF', 'Never Forgotten', 'Joe Bongiorno')]
[('0VOzdquK13dpkbgETqZdJz', 'Eagle', 'Dirk Maassen'), ('0Hs7MUnOvyQgymfXoEvm61', 'Week No. 11', 'Fabrizio Paterlini'), ('6ePBP1QrbQBrAJawrV8kiz', 

[('61liWcXv1YHijOqwXL1ki6', 'The Marriage - Finding Neverland/Soundtrack Version', 'Jan A.P. Kaczmarek'), ('3LFFoMyGHmiV3b5ZeaBF7O', 'Toys and Stars', 'Carter Burwell'), ('13kShSwCdr8BAK5pDBwIpA', 'Leaving Netherfield - From "Pride & Prejudice" Soundtrack', 'Dario Marianelli'), ('08WFl3fJEKm0WrJrYmGtkB', "Lueur d'été", 'Bruno Coulais'), ('2jpcucZ7Ap5j2hyQbDEGSn', 'Laputa: Castle In the Sky Suite', 'Eminence Symphony Orchestra')]
[('6Q00zs8KvmXWdwlHD7PZZ2', 'Vois sur ton chemin', 'Bruno Coulais'), ('2SFf2hOTXu6jnTZ9xSMepA', 'ナウシカ・レクイエム(風の谷のナウシカより)', 'Yuka'), ('5PyDwiyN0xzjSbdZ8bcUbd', 'Koe No Katachi - Lit', 'Theishter'), ('0VS7N1wCsGM2SeHGWW6Alo', 'HIGHLIGHTS FROM CASTLE IN THE SKY', 'Joe Hisaishi'), ('280HSUgzMhLc6a5eJMWGaR', 'Parker & Hachi Walk To The Station', 'Jan A.P. Kaczmarek')]
[('5Sd7U9BZtOqnxAUXnVCQSw', "Pachelbel's Canon Extreme", 'Kyle Landry'), ('2aIcanXxVk0tBtyWimy8Ew', 'Your Hands Are Cold - From "Pride & Prejudice" Soundtrack', 'Dario Marianelli'), ('08hlaHY7ssHzr2MvBK

...

[('2UqtDuGJkUa3jfyuBvt9Ma', 'Boss Bossanova', 'CrazyGroupTrio'), ('5K43oN6lgRfSd3HeQFdyG5', 'Puppet Ganon Snake Mode', 'Joy Vision'), ('4lQoDJdmR6L3JXKKvr2nuz', 'Onett', 'Mother 2'), ('74swD6kXoCw1R6e95HOXiJ', 'I\'m Just Your Problem (From "Adventure Time") [Orchestrated]', 'The Marcus Hedges Trend Orchestra'), ('0CkaWmFo1MRNDGf4n9lDNX', 'Turnabout Skeleton Brothers (Dating Start!) [Live]', 'insaneintherainmusic')]
[('5n8rcfpvV0cTEohMc5Tv3q', 'Star Trek V Main Title Theme - From "Star Trek Voyager"', 'Berlin Virtual Symphonics'), ('0CMUzgMnyoO5x3Z9xuCYxS', 'Great Fairy Fountain', 'Taylor Davis'), ('6j3KhHUQOC13nJKlAOX0gS', 'Crossing the first Treshold', 'Jonas B. Ingebretsen'), ('6Dyxoz14jd0lOkDjr7PNDV', 'Lavender Town', 'Braxton Burks'), ('5XSGmGkujWByM0VkAOWMhg', 'Character Theme Medley (From "Final Fantasy VI")', 'Various Artists')]
[('1bgwu34yKiux9GmsO1LSSa', 'Fairy Tale - From Inuyasha', 'RMaster'), ('28FVT63cVVmVLqHCDuMnqX', 'City Under Siege', '3000m'), ('0oxIAvNwaoTl0xpISzzCON'

[('1MhsppnDeFdr6pj4HqhXLk', 'Ace and Deuce of Pipering@108', 'Billy Furlong'), ('565WZnoWcf2zQYa5qnAR8T', 'The Gates Of Derry', 'Sean Quinn Band'), ('6Vx5omwGTsMJiLmZESU06B', 'The Merry Ploughboy', 'The Wolfe Tones'), ('5b86vvW8lwSC659psOCT58', 'Planxty Davis 109', "Sean O'Brien"), ('5Y9CMy46m1KvBm6kZags2b', 'Novice Reels (116)', 'David Lindquist')]
[('1aujAnZcHk64X5lMC3fr9x', 'Beginner Reel (121)', 'William Paterson'), ('21agKdX3vDyFvzgkW9zN88', 'Beginner Single Jigs: Scattery Island / Lithium Liberace / The Humours of Limerick', 'Feistunes'), ('0swcDLyF6kB9b2qg4Zjsx7', 'Trip to the Cottage', 'Chris and Michelle McLoughlin'), ('6Pyv1lF1mwDMBqZlZ6Oywl', 'Ardghrad / Craobhghrad Slip Jig, Pt. 2', 'Darren Checkley'), ('6HRZx4SeEG3G6Luv0UnfQl', 'The Piper 76', "Sean O'Brien")]
[('6x8xjV62IPx7x31Bvxej4B', 'Castle Kelly / Galway Rambler', 'Liz Carroll'), ('0tsEIHQnnwHF19SyJgad1K', "A Kerry Reel/Michael Coleman's/The Wheels Of The World/Julia Delaney", 'Kevin Burke'), ('4TTekCUiskYs0u8jil3pX3

[('3EQtcJwaWTMrvXJ6Om8Ogx', 'Shout Out to My Ex', 'Declan Wilson'), ('4xKvFAGYX6tU2qJ2dECjU0', 'Ardghrad / Craobhghrad Single Jig 116', 'William Paterson'), ('3g3VOK3VVFzzf6223YjsTS', 'Back in the Lab', 'Beoga'), ('1CY1IvbK3chbdBleJUh0hv', 'Primary Light Jigs, Pt. 1', 'Gerry Conlon'), ('0B7ho8FZI0fOoEcwC3P0Pg', 'Traditional Heavy Jig', 'Anton & Sully')]
[('6nPBGxeDSi0nm8pNHLlYb3', 'Michael Conway', 'Solas'), ('1te7cCyr9QSqZXyumP8DWB', 'Starry Slip Jigs 113', 'Ellery Klein & Ryan Lacey'), ('73p6sseN85FIigv7hdc4MB', "Jerome Lacey / The Rogue's Reel", 'Liz Carroll'), ('0lNtdZNN7GE3LocZZ4Vw62', 'Madame Bonaparte', 'Jim Butke & Merv Bell'), ('4xbqWr34R29hgsdtyje2N8', 'Treble Reels 118 (sped-up reels from the CD Champtrax)', 'Mark Arrington')]
[('7dquIJCv0sE6PibVHsrw7v', "Me Traveller's Daughter", 'Seamus Moore'), ('1hqn1dEnR856qPngkBG5Rf', "Ireland's Green Shore", 'Goitse'), ('4wpMhcja66IbLT6igDluhx', 'The Waves Of Tory', 'Sean Quinn Band'), ('1zVzXGC0ae2ihP3Or9PFba', 'Jig Pt 1', 'Colm Keog

[('6hIvTla9fTUUrDkXSKzIFg', 'Scupper the Plan', 'Michael Fitzpatrick'), ('7p4cxMPMvoIfjjrv8L4rPI', 'Reel Beatrice', 'Sharon Shannon'), ('2M2pgj6VPOkviGXpObqHKG', 'The Jacqueline (Walzer)', 'Various Artists'), ('4DPIDiqzzJE4wkCxhVW0eM', 'Jig Pt 2', 'Colm Keogh'), ('0F5ONZGIurSVQsxrmKrT7E', 'The Spanish Lady', 'The Dubliners')]
[('2CiVnYjUAv1cUe9pOMUH6s', 'The Ace and Deuce of Pipering (BPM 76)', 'Pat King'), ('3iuQCxLrlhwkuFsDems1zn', 'Primary Treble Jigs 85', 'Mark Arrington'), ('0lNtdZNN7GE3LocZZ4Vw62', 'Madame Bonaparte', 'Jim Butke & Merv Bell'), ('147POzCF7W1QCjQw3FANah', 'Reel 113, Pt. 1', 'Darren Checkley'), ('0WDA562M4VgoJWbWzlpBq3', 'Reel Beatrice, The Old Cambridge Hornpipe', "Gerry Conlon & Seamus O'Sullivan")]
[('2sFfTuFdgDLkUgbg6D8uOp', "Hop Jigs (Monkey Wriggle / Lep the Table / Thadelo's)", 'Tina Jordan Rees'), ('5XsKPD9IjeYKZ2k1MPEx5l', 'Heavy Jigs 1 (73) Carbonado / Jig for Jimmy', "Sean O'Brien"), ('0tqHB1opjUUtpTWGqO81y6', 'Is It Tea You Want, Tipperary Hills', "Gerry

[('5c7dCn3P0WhdvwHsB0kOfp', 'Meanghrad Single Jig 122', 'William Paterson'), ('3V5jQQOoBxzZClmielN0ZF', "Johnny Barry's Helicopter", 'Richie Kavanagh'), ('6i0O6gBeGe86sq7LFOkhMn', 'Beginners Hornpipe', 'Colm Keogh'), ('0ioSSSaJavAKbvA6OGhXBR', 'Meangrhad Reel, Pt. 2', 'Darren Checkley'), ('0pwwhmo6beqOu1ZLUU5iym', 'Single/Hop Jigs', 'Chris and Michelle McLoughlin')]
[('1awM3eh6hj4R1VPkBmqhi5', 'Meanghrad Slip Jig, Pt. 1', 'Darren Checkley'), ('3TbW8nlDIkozRIAnTVSsYU', "King's Favourite, Scottish Jig", "Gerry Conlon & Seamus O'Sullivan"), ('15PM2ZDhVHE3XMHhdGREQ4', 'Boxy Set', 'Beoga'), ('0swcDLyF6kB9b2qg4Zjsx7', 'Trip to the Cottage', 'Chris and Michelle McLoughlin'), ('5DPibvfzEfKax2lCbHqsm0', 'Ace and Deuce of Pipering (BPM 110)', 'Pat King')]
[('5byJTFmhJt142qTxoA6yCY', 'Music in the Glen Reels 113 (Music in the Glen / Pigeon On the Gate / The Pizza Reel)', 'Ellery Klein & Ryan Lacey'), ('2x1mGZF7PzC3u5F1HsSIFC', 'Last House in the Village', 'Michael McGoldrick'), ('7t0zr4P4UDJ42kiS

Now, for each recommended song we have to extract audio features and store them in a data frame. **Implement**

Finally, we can filter out the recommendations using our classifier, and get the songs which most probably we can like. **Implement**

In [44]:
rec_tracks

['6HZBLw5O3bowdv1JpNNDRw',
 '3Lgq7X51hHSTJ8njTAlUmb',
 '1nd37vT8JNshn5Jrq0KN0M',
 '0wf136XdLOC4tMSnhTDabf',
 '32Lw7lCpUZsecY3gGeORxf',
 '30X3fQ87XRzIilgWrYVBd4',
 '0qTb6APm0CI4DGFfLQJS20',
 '0g1xBJGgVnL8tUEIoH0ng9',
 '0uFwdpkVJzpyUxlu9reTWK',
 '6Zpn04HhXbuU9sLJQIY0bI',
 '11Gj7oeUz27BskXHwDsWH1',
 '2MIVLgLJcfAKB8CGutZPhY',
 '66LFlaSMlWNEIHbBbkzxhD',
 '6pcIgvJnBC724thDZDdQsC',
 '3vVb5JqcEqb1UdM24oYSSq',
 '1gGW9wFECrRX33yDvAqCSw',
 '6wX6saGLqTalPCXHf69sr1',
 '2YAHBaGNPxszDSKNQhOvTv',
 '6XdVpsH2Es9Gh7f6UPyBwU',
 '093oXyRf76B4dL3G350Yb5',
 '5CH4S8zHiW2I7TpNhn6a4v',
 '4XJrzixx8cw8JpI1I64llY',
 '3Ny6E1kTHJ7Sc3u7DvlFY7',
 '36ZsqQpdF4yRhb8XJV6aEs',
 '7r8zqoOauLuClfKbx7zcfP',
 '3YYUmw4asVeqUlkJMKPfzW',
 '5XshbHd16oFL3hRzxE72Xq',
 '5VEVnqQu6cFewcbWfbkUeE',
 '3WXihZyREYjtzDjTfArJN2',
 '3Xi4Hu8BcUNdUBmBjpkh20',
 '1KP0YO8oWOzpef53wIrFDL',
 '5fUKjWT9DXC1yYRnvumMXe',
 '26krqFBg6lylTse1mtW5Ef',
 '0xbUb9AjCr9yASgGTqhZWu',
 '0jNhSkUv9Xx8ezu4ESZ52Y',
 '08evqQGBMQRDH6jUWPos6P',
 '6rL9JBm6WmXVoiz8sVgFzH',
 